# Optimization


In this exercise you will be asked to speed up a finite-difference wavefield modeling code. You will be given a briefing of the theoretic background of finite-difference modeling and a working code, however the provided code is very inefficient and can be improved significantly in many aspects. Your goal is to find out all possible speed-ups and make it as fast as you can.


# Introduction to finite-difference wavefield modeling

Wavefield modeling is a fundamental tool used in geophysics. 
Many advanced imaging and velocity analysis techniques require wave propagation.
There are many ways to extrapolate wavefields. They vary in computational difficulty and accuracy.
In this lab, you will be introduced to the basics of wave propagation using the time-domain finite-difference (TDFD) technique. 

## Time-domain finite difference (TDFD)

The following shows the acoustic wave equation for an isotropic constant-density medium:

\begin{equation}
\frac{\partial^2 P(x,z,t)}{\partial t^2} = v^2 \nabla^2 P(x,z,t) + f(x,z,t),
\end{equation}

where $P(x,z,t)$ represents the wavefield in time $t$ and at subsurface location $(x,z)$, $f(x,z,t)$ is the source term that injects the initial energy into the propagation domain. In our case, the point source is very compact in time and space, meaning that $f(x,z,t)$ is mostly zero, we will ignore $f$ for this moment.

Notice there is a second derivative in time ($\frac{\partial^2 }{\partial t^2}$) and a Laplacian ($\nabla^2=(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial z^2} ) $) in space.
In TDFD, we discretize the wavefield in time $t=\{t_k\}$ and space $(x,z)=\{x_i,z_j\}$ with the notation $P_{ij}^{k}$.
If a second-order approximation is used for the time derivative, we get the following time-marching equation:

\begin{equation}
\frac{(P_{ij}^{k-1} - 2P_{ij}^{k} +  P_{ij}^{k+1})}{\triangle t^2} = v^2  \nabla^2 P_{ij}^{k} \nonumber 
\end{equation}
\begin{equation}
P_{ij}^{k+1} =  - P_{ij}^{k-1} + 2P_{ij}^{k} + v^2 \triangle t^2 \nabla^2 P_{ij}^{k}.
\end{equation}

\noindent The time-marching equation is the basis of wavefield extrapolation in the time-domain. 
To calculate the wavefield for the next time slice ($P_{ij}^{k+1}$), you use the information from the current ($P_{ij}^{k}$) and the previous time slices ($P_{ij}^{k-1}$). 
Besides the time-derivative, the Laplacian ($\nabla^2$) will need to be approximated with finite-differences. 
To get a lower numerical error, people often use a higher order approximation for the spatial derivatives.
For example, the 10th order finite-difference approximation will use 11 points along each axis to compute the Laplacian operator. i.e:

\begin{eqnarray*}
 \frac{\partial^2}{\partial x^2}P_{i,j} & = & \frac{1}{\triangle x^2} * \big( \\
 &  & c_{-5}P_{i-5,j} + c_{5}P_{i+5,j} + \\
 &  & c_{-4}P_{i-4,j} + c_{4}P_{i+4,j} + \\
 &  & c_{-3}P_{i-3,j} + c_{3}P_{i+3,j} + \\
 &  & c_{-2}P_{i-2,j} + c_{2}P_{i+2,j} + \\
 &  & c_{-1}P_{i-1,j} + c_{1}P_{i+1,j} + \\
 &  & c_{0}P_{i,j} \\
 &  & \big).
\end{eqnarray*}
Similarly for $z$ axis. The coefficients $c_m$ are carefully crafted such that they minimizes the approximation error. These coefficients are provided to you in the code.

# Finite-difference modeling code optimization.
You are given `prop.cc` and `field.cc` to start with. This code performs the TDFD wavefield modeling and reports the time of computation.
First, you should figure out how the code does it, and starting from there, modify the code to gradually reduce its inefficiency. **Remember to mark all changes you made and write down the new run-time every time you are adding an improvement.**

Below you will find the files to edit.

In [ ]:
%%writefile src/field3d.h
#pragma once
#include <vector>
#include <string>
#include <iostream>
#include <stdio.h>

typedef std::vector<std::vector<std::vector<float>>> mat3d;

class field3d {
 public:
  field3d() {}
  field3d(const int n1, const int n2, const int n3);
  field3d(const int n1, const int n2, const int n3, const float val);
  field3d(const field3d &f);
  field3d(const int n1, const int n2, const int n3,
          const std::vector<int> pos, const std::vector<int> off1,
          const std::vector<int> off2, const std::vector<int> off3,
          const std::vector<float> vals);
  field3d &operator*=(const field3d &a) {
    std::vector<int> ns = a.getSize();
    const mat3d as = a.getCVals();
    allocate(ns[0], ns[1], ns[2]);
    for(int i3 = 0; i3 < ns[2]; i3++) {
      for(int i2 = 0; i2 < ns[1]; i2++) {
        for(int i1 = 0; i1 < ns[0]; i1++) {
          _vals[i1][i2][i3] *= as[i1][i2][i3];
        }
      }
    }
    return *this;
  }
  float sum() const;
  field3d &operator+=(const field3d &a) {
    fprintf(stderr, "in += \n");
    std::vector<int> ns = a.getSize();
    const mat3d as = a.getCVals();
    allocate(ns[0], ns[1], ns[2]);
    for(int i3 = 0; i3 < ns[2]; i3++) {
      for(int i2 = 0; i2 < ns[1]; i2++) {
        for(int i1 = 0; i1 < ns[0]; i1++) {
          _vals[i1][i2][i3] += as[i1][i2][i3];
        }
      }
    }
    return *this;
  }
  const mat3d getCVals() const {
    return _vals;
  }
  mat3d getVals() {
    return _vals;
  }
  std::vector<int> getSize() const;
  void allocate(const int n1, const int n2, const int n3);

 private:
  mat3d _vals;
};

field3d operator+(const field3d a, const field3d b);
field3d operator-(const field3d a, const field3d b);
field3d operator*(const field3d a, const field3d b);
field3d operator*(const field3d a, const float b);
field3d operator*(const float b, const field3d a);

In [ ]:
%%writefile src/field3d.cc
#include <field3d.h>
#include <stdio.h>
#include <iostream>

field3d::field3d(const int n1, const int n2, const int n3) {
  allocate(n1,n2,n3);
}

void field3d::allocate(const int n1, const int n2, const int n3) {
  std::vector<float> v1(n3, 0.);
  std::vector<std::vector<float>> v2(n2, v1);
  _vals.resize(n1, v2);     
}

field3d::field3d(const int n1, const int n2, const int n3, const float val) {
  allocate(n1,n2,n3);
  for(int i3 = 0; i3 < n3; i3++) {
    for(int i2 = 0; i2 < n2; i2++) {
      for(int i1 = 0; i1 < n1; i1++) {
        _vals[i1][i2][i3] = val;
      }
    }
  }
}

field3d::field3d(const field3d&f) {
  mat3d vals = f.getCVals();
  std::vector<int> ns = f.getSize();
  int n1 = vals.size();
  int n2 = vals[0].size();
  int n3 = vals[0][0].size();
  
  allocate(n1, n2, n3);
  for(int i3 = 0; i3 < n3; i3++) {
   for(int i2 = 0; i2 < n2; i2++) {
     for(int i1 = 0; i1 < n1; i1++) {
        _vals[i1][i2][i3] = vals[i1][i2][i3];
      }
    }
  }
}

std::vector<int> field3d::getSize() const {
  std::vector<int> ns;
  ns.push_back(_vals.size());
  ns.push_back(_vals[0].size());
  ns.push_back(_vals[0][0].size());
  return ns;
}

field3d::field3d(const int n1, const int n2, const int n3, 
                 const std::vector<int> ipos, const std::vector<int> off1,
                 const std::vector<int> off2, const std::vector<int> off3,
                 const std::vector<float> vals) {
  allocate(n1, n2, n3);
  for(int i = 0; i < off1.size(); i++)
    _vals[off1[i]+ipos[0]][off2[i]+ipos[1]][off3[i]+ipos[2]] = vals[i];
}

field3d operator-(const field3d a, const field3d b) {
  std::vector<int> ns = a.getSize();
  const mat3d as = a.getCVals(), bs = b.getCVals();
  field3d c(ns[0], ns[1], ns[2]);
  mat3d cs = c.getVals();
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0; i2 < ns[1]; i2++) {
      for(int i1 = 0; i1 < ns[0]; i1++){
        cs[i1][i2][i3] = as[i1][i2][i3] - bs[i1][i2][i3];
      }
    }
  }
  return c;
}

field3d operator+(const field3d a, const field3d b) {
  std::vector<int> ns = a.getSize();
  const mat3d as = a.getCVals(), bs = b.getCVals();
  field3d c(ns[0], ns[1], ns[2]);
  mat3d cs = c.getVals();
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0; i2 < ns[1]; i2++) {
      for(int i1 = 0; i1 < ns[0]; i1++) {
        cs[i1][i2][i3] = as[i1][i2][i3] + bs[i1][i2][i3];
      }
    }
  }
  return c;
}

float field3d::sum() const {
  std::vector<int> ns = getSize();
  float sm = 0;
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0;i2 < ns[1]; i2++) {
     for(int i1 = 0;i1 < ns[0]; i1++) {
        sm += _vals[i1][i2][i3];
      }
    }
  }
  return sm;
}

field3d operator*(const field3d a, const field3d b){
   std::vector<int> ns=a.getSize();
   const mat3d as=a.getCVals(),bs=b.getCVals();
   field3d c(ns[0],ns[1],ns[2]);
   mat3d cs=c.getVals();
      for(int i3=0; i3 < ns[2]; i3++){
     for(int i2=0; i2 < ns[1]; i2++){
       for(int i1=0; i1 < ns[0]; i1++){
          cs[i1][i2][i3]=as[i1][i2][i3]*bs[i1][i2][i3];
        }
      }
    }
    return c;
}
field3d operator*(const field3d a, const float b) {
  std::vector<int> ns = a.getSize();
  const mat3d as = a.getCVals();
  field3d c(ns[0], ns[1], ns[2]);
  mat3d cs = c.getVals();
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0; i2 < ns[1]; i2++) {
      for(int i1 = 0; i1 < ns[0]; i1++) {
        cs[i1][i2][i3] = as[i1][i2][i3] * b;
      }
    }
  }
  return c;
}

field3d operator*(const float b, const field3d a) {
  std::vector<int> ns = a.getSize();
  const mat3d as = a.getCVals();
  field3d c(ns[0], ns[1], ns[2]);
  mat3d cs = c.getVals();
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0; i2 < ns[1]; i2++) {
      for(int i1 = 0; i1 < ns[0]; i1++) {
        cs[i1][i2][i3] = as[i1][i2][i3] * b;
      }
    }
  }
  return c;
}

In [ ]:
%%writefile  src/prop.h
#pragma once
#include <field3d.h>

class vel3d: public field3d {
 public:
  vel3d() {}
  vel3d(const int n1, const float d1, const int n2, const float d2,
        const int n3, const float d3, float vconst);  
  vel3d(const vel3d &v);
  float getD1()const {return _d1;}
  float getD2()const {return _d2;}
  float getD3()const {return _d3;}
  
 private:
  float _d1, _d2, _d3;
};

class prop {
 public:
  prop(const float dt, const vel3d vel, std::vector<int> isrc, 
       std::vector<float> coef);
  field3d advance(const field3d prev, const field3d cur, const int it) const;
  field3d applyLaplacian(const field3d cur) const;
  field3d sourceTerm(const int it) const;
  
 private:
  vel3d _vel;
  float _dt;
  std::vector<int> _isrc, _off1, _off2, _off3;
  std::vector<float> _coef;
};

In [ ]:
%%writefile src/prop.cc
#include <prop.h>
#include <field3d.h>
#include <assert.h>
#include <math.h>
#include <stdio.h>
#include <timer.h>

vel3d::vel3d(const int n1, const float d1, const int n2, const float d2,
             const int n3, const float d3, float vconst) {
  allocate(n1, n2, n3);
  mat3d cs = getVals();
  for(int i3 = 0; i3 < n3; i3++) {
    for(int i2 = 0; i2 < n2; i2++) {
      for(int i1 = 0; i1 < n1; i1++) {
        cs[i1][i2][i3] = vconst;
      }
    }
  }     
  _d1 = d1;
  _d2 = d2;
  _d3 = d3;
}

vel3d::vel3d(const vel3d &v) {
  std::vector<int> ns = v.getSize();
  allocate(ns[0], ns[1], ns[2]);
  mat3d cs = getVals();
  const mat3d as = v.getCVals();
  for(int i3 = 0; i3 < ns[2]; i3++) {
    for(int i2 = 0; i2 < ns[1]; i2++) {
      for(int i1 = 0; i1 < ns[0]; i1++) {
        cs[i1][i2][i3] = as[i1][i2][i3];
      }
    }
  }     
  _d1 = v.getD1();
  _d2 = v.getD2();
  _d3 = v.getD3();
}

prop::prop(const float dt, const vel3d v, const std::vector<int> isrc,
           const std::vector<float> coef) {
  _vel = vel3d(v);
  _dt = dt;
  _isrc = isrc;
  assert(coef.size() == 5);
  std::vector<int> ns = v.getSize();
  float sm = 0;
  for(int i = 0; i < 5 ; i++) {
    _off1.push_back(i); _off2.push_back(0); _off3.push_back(0);
    _coef.push_back(coef[i]);
    _off1.push_back(0); _off2.push_back(i); _off3.push_back(0);
    _coef.push_back(coef[i]);
    _off1.push_back(0); _off2.push_back(0); _off3.push_back(i);
    _coef.push_back(coef[i]);
    _off1.push_back(-i); _off2.push_back(0); _off3.push_back(0);
    _coef.push_back(coef[i]);
    _off1.push_back(0); _off2.push_back(-i); _off3.push_back(0);
    _coef.push_back(coef[i]);
    _off1.push_back(0); _off2.push_back(0); _off3.push_back(-i);
    _coef.push_back(coef[i]);
    sm += coef[i];
   }
   _off1.push_back(0); _off2.push_back(0); _off3.push_back(0);
   _coef.push_back(-2. * sm);
}

field3d prop::applyLaplacian(const field3d cur) const {
  std::vector<int> ns = cur.getSize();
  field3d lap(ns[0], ns[1], ns[2]);
  std::vector<int> pos(3, 0);
  mat3d l = lap.getVals();
  timer tm(std::string("laplacian"), (ns[0]-10));
  for(int i3 = 5; i3 < ns[2]-5; i3++) {
    for(int i2 = 5; i2 < ns[1]-5; i2++) {
      tm.start();
      for(int i1 = 5; i1 < ns[0]-5; i1++) {
        pos[0] = i1; pos[1] = i2; pos[2] = i3;
        field3d filt(ns[0], ns[1], ns[2], pos, _off1, _off2, _off3, _coef);
        field3d temp(cur);
        temp = filt * cur;
        l[i1][i2][i3] = temp.sum();
      }
      tm.stop();
      tm.print();
    }
  }
  return lap;
}

field3d prop::sourceTerm(const int it) const {
  std::vector<int> ns = _vel.getSize(); 
  field3d source(ns[0], ns[1], ns[2]);
  mat3d s = source.getVals();
  s[_isrc[0]][_isrc[1]][_isrc[2]] = sin(it * 3.1415926536 / 20.);
  return source;
}

field3d prop::advance(const field3d cur, const field3d prev, const int it) const {
  mat3d c = cur.getCVals(), p = prev.getCVals();
  field3d spaceDeriv = applyLaplacian(cur) * _vel * _vel;
  field3d next(cur);
  next = 2. * cur - prev + (_dt * _dt * spaceDeriv) + sourceTerm(it);
  return next;
}

In [ ]:
%%writefile src/main.cc
#include <field3d.h>
#include <prop.h>
#include <stdio.h>

int main(int argc, char **argv) {
  int n1 = 100, n2 = 100, n3 = 100;
  float vconst;
  vel3d vel(n1, .01, n2, .01, n3, .01, vconst);
  std::vector<float> coef(5, 0.);
  coef[0] =  .83333333;
  coef[1] = -.119047619;
  coef[2] =  .0198412698;
  coef[3] =  .00248015873;
  coef[4] =  .0001587301587;

  field3d prev(n1, n2, n3);
  field3d cur(n1, n2, n3);
  std::vector<int> isrc(3, 250);
  prop p(.002, vel, isrc, coef);

  for(int it = 0; it < 500; it++) {
    fprintf(stderr, "working on iteration %d \n", it);
    field3d next = p.advance(prev, cur, it);
    prev = cur;
    cur = next;
  }
}

Below you will find the `CMakeLists.txt` file and how to compile and run the code.

In [ ]:
%%writefile src/CMakeLists.txt
#TELL WHAT VERSION OF CMAKE WE REQUIRE
cmake_minimum_required(VERSION 2.8 FATAL_ERROR)

#Name our project and language we are going to use
project(example LANGUAGES CXX)

#We want to require C++11
set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUIRED ON)

include_directories(${CMAKE_CURRENT_SOURCE_DIR})

#Here is the executable we want to build
add_executable(prop.x prop.cc main.cc field3d.cc timer.cc)

In [ ]:
!mkdir -p build; cd build; rm -rf *; cmake -DCMAKE_CXX_FLAGS="-O0" ../src; make

In [ ]:
!./build/prop.x

# What to show 

Explain all of your changes and the speedup each change made. Estimate your reads per floating point operator.  Based on the machine you are running on make a roofline plot and figure out if you are memory or compute bound? Plot your performance on your roofline model.